In [1]:
import pickle

In [8]:
# open a file, where you stored the pickled data
file = open('wine_word2vec_model', 'rb')

# dump information to that file
wine_word2vec_model = pickle.load(file)

# close the file
file.close()

# Same for the rest of the files
file = open('model_knn', 'rb')
model_knn = pickle.load(file)
file.close()

file = open('dict_of_tfidf_weightings', 'rb')
dict_of_tfidf_weightings = pickle.load(file)
file.close()

file = open('wine_reviews_mincount', 'rb')
wine_reviews_mincount = pickle.load(file)
file.close()

In [9]:
def descriptors_to_best_match_wines(list_of_descriptors, number_of_suggestions=10):
    weighted_review_terms = []
    for term in list_of_descriptors:
        if term not in dict_of_tfidf_weightings:
            if term not in descriptor_mapping.index:
                print('choose a different descriptor from', term)
                continue
            else:
                term = descriptor_mapping['normalized'][term]
        tfidf_weighting = dict_of_tfidf_weightings[term]
        word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
        weighted_word_vector = tfidf_weighting * word_vector
        weighted_review_terms.append(weighted_word_vector)
    review_vector = sum(weighted_review_terms)

    distance, indice = model_knn.kneighbors(review_vector, n_neighbors=number_of_suggestions+1)
    distance_list = distance[0].tolist()[1:]
    indice_list = indice[0].tolist()[1:]

    n = 1
    for d, i in zip(distance_list, indice_list):
        wine_name = wine_reviews_mincount['title'][i]
        wine_descriptors = wine_reviews_mincount['normalized_descriptors'][i]
        print('Suggestion', str(n), ':', wine_name, 'with a cosine distance of', "{:.3f}".format(d))
        print('This wine has the following descriptors:', wine_descriptors)
        print('')
        n+=1

In [10]:
descriptors = ['apple','pear','wood']
descriptors_to_best_match_wines(list_of_descriptors=descriptors, number_of_suggestions=5)

Suggestion 1 : Domaine Faiveley 2011 Les Villeranges  (Rully) with a cosine distance of 0.084
This wine has the following descriptors: ['fresh', 'fruit', 'citrus', 'pear', 'apple', 'wood', 'round']

Suggestion 2 : Domaine Ostertag 2010 Barriques Pinot Blanc (Alsace) with a cosine distance of 0.088
This wine has the following descriptors: ['oak', 'wood', 'toast', 'pear', 'honey', 'apple']

Suggestion 3 : A.R. Lenoble  NV Intense Brut  (Champagne) with a cosine distance of 0.110
This wine has the following descriptors: ['wood', 'rich', 'ripe', 'fruit', 'pear', 'crisp', 'apple', 'apricot']

Suggestion 4 : Domaine de la Garenne 2008  Mâcon-Azé with a cosine distance of 0.120
This wine has the following descriptors: ['wood', 'fruit', 'fresh', 'grapefruit', 'pear', 'bright', 'lemon', 'honey', 'wood']

Suggestion 5 : Wheelhouse 2011 Chardonnay (Sonoma County) with a cosine distance of 0.126
This wine has the following descriptors: ['ripe', 'apple', 'pear', 'tropical_fruit', 'fruit', 'oak']



In [11]:
wine_reviews_mincount

,level_0,index,country,description,designation,points,price,province,region_1,title,variety,winery,normalized_descriptors,review_vector,descriptor_count
0,0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,"[tropical_fruit, fruit, dry, herb, apple, citr...","[[-0.37825504, -5.1521606, -1.7322228, -2.4307...",9
1,1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,"[ripe, fruit, smooth, firm, juicy, berry, frui...","[[-1.2453364, -3.138025, 1.7377969, -0.0509741...",8
2,2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,"[tart, snappy, lime, green, pineapple, crisp, ...","[[-5.1481357, -4.3782115, 0.27906758, 0.112504...",7
3,3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,"[pineapple, rind, lemon_pith, orange_blossom, ...","[[-1.2961392, -2.4018247, 1.1718184, 0.2752304...",6
4,5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,"[blackberry, raspberry, green, herb, full_bodi...","[[-1.6322763, -2.6095781, 1.1783365, -2.78001,...",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108872,129965,129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,"[rich, dry, fresh, ripe, white, fruit, pear, a...","[[-1.0220966, -3.155178, -0.23385222, 0.952300...",9
108873,129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),"[honeysuckle, cataloupe, sweet, juicy, tart, t...","[[-4.227007, -3.1213841, 0.016722424, 2.088082...",9
108874,129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,"[baked, cherry, cocoa, coconut, soft, fruit, c...","[[-0.38147363, -3.3817122, 2.0994532, -3.18786...",7
108875,129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,"[dry, crisp, weight, powerful, spice, baked_ap...","[[-1.0713768, -2.2569797, 3.7971995, 1.7632971...",6


In [12]:
rv = wine_reviews_mincount.review_vector

In [20]:
rv[0][0].dtype

dtype('float32')